## Train step 1: Bootstrap from pretrained model

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2
%matplotlib notebook

import pickle, os, warnings, sys, random, logging, librosa, json, nemo
warnings.filterwarnings("ignore")
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from Cfg import Cfg
from reshuffle_samples import reshuffle_samples
import nemo.collections.asr as nemo_asr
from ruamel.yaml import YAML

from omegaconf import DictConfig
import os, datetime
from load_pretrained_amharic_model import load_pretrained_amharic_model

from reshuffle_samples import reshuffle_samples
import pytorch_lightning as pl

[NeMo W 2020-10-15 18:36:36 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-15 18:36:36 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-15 18:36:36 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-15 18:36:36 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text._TarredAudioToTextDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-15 18:36:36 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.TarredAudioToCharDataset'> is experimental,

In [2]:
C, model = load_pretrained_amharic_model()

TypeError: load_pretrained_amharic_model() missing 1 required positional argument: 'C'

In [ ]:
config_path = 'amharic_16000.yaml'
yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
train_manifest=f'{C.build_dir}/train_manifest.json'
test_manifest=f'{C.build_dir}/test_manifest.json'
params['model']['train_ds']['manifest_filepath'] = train_manifest
params['model']['validation_ds']['manifest_filepath'] = test_manifest

model.setup_training_data(train_data_config=params['model']['train_ds'])
model.setup_validation_data(val_data_config=params['model']['validation_ds'])

In [ ]:
model_save_dir='save/nemo_amharic'

class ModelCheckpointAtEpochEnd(pl.callbacks.ModelCheckpoint):
    def on_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        metrics['epoch'] = trainer.current_epoch
        trainer.checkpoint_callback.on_validation_end(trainer, pl_module)

pid=os.getpid()
dt=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

checkpoint_callback = ModelCheckpointAtEpochEnd(
    filepath=model_save_dir+'/amharic_'+f'{dt}_{pid}'+'_{epoch:02d}',
    verbose=True,
    save_top_k=-1,
    save_weights_only=False,
    period=1)

In [ ]:
trainer = pl.Trainer(gpus=[0], max_epochs=200, amp_level='O1', precision=16, checkpoint_callback=checkpoint_callback)

In [ ]:
params['model']['optim']['lr']=0.001

In [ ]:
model.setup_optimization(optim_config=DictConfig(params['model']['optim']))

In [ ]:
reshuffle_samples(C)

In [ ]:
trainer.fit(model)